In [1]:
import pandas as pd
import numpy as np
import os
import json

In [3]:
main_model_dir="/home/hrenduchinta/LM-pretraining/models/fl_bert_07_10_2022_05:55:01"
for step in range(50000, 300000, 50000):
    model_path=f"{main_model_dir}/step_{step}"
    try:
        data=pd.DataFrame(columns=["cb", "copa", "wsc", "boolq", "multirc", "wic"], index=[f"superglue_run_{i}" for i in range(1, 21)])
        task2metric={"boolq":"acc", "cb": "avg_f1", "copa":"acc", "multirc":"f1", "record":"f1", "wic": "acc", "wsc": "acc"}
        for run in range(1, 21):
            if run>5:
                tasks=["cb", "copa", "wsc"]
            else:
                tasks=["cb", "copa", "wsc", "boolq", "multirc", "wic"]
            l=[]
            for task in tasks:
                with open(f"{model_path}/superglue_run_{run}/runs/{task}/val_metrics.json") as f:
                    d=json.load(f)
                    score=d[task]["metrics"]["minor"][task2metric[task]]
                    l.append(round(100.0*float(score), 2))
            if run>5:
                l.extend([np.nan]*3)
            data.loc[f"superglue_run_{run}"]=l
        data.loc["average"]=data.mean(axis=0, skipna=True)
        data.loc["stddev"]=data.std(axis=0, skipna=True)
        data.loc["decrement_avg"]=[np.nan for i in range(6)]
        data.loc["maximum"]=data.max(axis=0, skipna=True)
        data.loc["decrement_max"]=[np.nan for i in range(6)]
        data.to_csv(f"{model_path}/superglue_results.csv")
    except Exception as e:
        print(f"step-{step}: {str(e)}")

In [ ]:
main_model_dir="/home/hrenduchinta/LM-pretraining/models/fl_without_warmstart_bert_11_06_2022_17:53:56"
for step in range(50000, 300000, 50000):
    for run in range(1, 21):
        if run>5:
            tasks=["cb", "copa", "wsc"]
        else:
            tasks=["cb", "copa", "wsc", "boolq", "multirc", "wic"]
        for task in tasks:
            if not os.path.isfile(f"{main_model_dir}/step_{step}/superglue_run_{run}/runs/{task}/val_metrics.json"):
                print(step, run, task)

In [2]:
import json
from statistics import mean, stdev

In [24]:
l=[]
for i in range(1,21):
    with open(f"/home/hrenduchinta/LM-pretraining/models/regular_bert/test_superglue_run_{i}/runs/cb/val_metrics.json", "r") as f:
        d=json.load(f)
        l.append(100*d["cb"]["metrics"]["minor"]["avg_f1"])

In [25]:
mean(l), stdev(l)

(74.88759057452789, 2.4532846869924643)

In [9]:
l=[]
for i in range(1,6):
    with open(f"/home/hrenduchinta/LM-pretraining/models/regular_bert/test_superglue_run_{i}/runs/copa/val_metrics.json", "r") as f:
        d=json.load(f)
        l.append(100*d["copa"]["metrics"]["minor"]["acc"])

In [10]:
mean(l), stdev(l)

(67.8, 3.3466401061363023)

In [11]:
l

[70.0, 62.0, 68.0, 70.0, 69.0]